# KONEX 크롤링
- 주의: Selenium, webdrive-manager 가 다운되어있어야 한다!
- API보다 사용하기 쉽고 직관적이라는 장점이 있으나, 더럽게 오래걸린다는 단점이 있다.
- 또한, Jquerry가 아닌 JavaScript 기반이라면 더더욱 어렵다. 

In [78]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException, ElementClickInterceptedException, TimeoutException
import time

row = []
corp_names = []
listed_time = []

# 웹 드라이버 경로 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 웹 페이지 열기
driver.get("https://kind.krx.co.kr/listinvstg/bzrsltdrft.do?method=searchBzRsltDrftMain")
time.sleep(3)

# 코넥스 시장 선택
button1 = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f'/html/body/section[2]/section/form/section/div/div[1]/table/tbody/tr/td[1]/input[4]'))  # 버튼의 XPATH로 변경
)
button1.click()
time.sleep(1)

# 코넥스 시장 검색
button2 = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f'/html/body/section[2]/section/form/section/div/div[3]/a[1]'))  # 버튼의 XPATH로 변경
)
button2.click()
time.sleep(1)

# 특정 버튼을 기다리고 클릭하기
for j in range(1,10): #10
    time.sleep(3)
    for i in range(1, 16): #16
        try:
            # 각 기업명 누르기
            button3 = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f'/html/body/section[2]/section/div/article/section[1]/table/tbody/tr[{i}]/td[1]/a'))  # 버튼의 XPATH로 변경
            )
            button3.click()

            corp = button3.text
            corp_names.append(corp)

            # 새로운 창으로 전환하기
            WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))  # 새로운 창이 열릴 때까지 대기
            original_window = driver.current_window_handle  # 원래 창 핸들 저장
            new_window = [window for window in driver.window_handles if window != original_window][0]  # 새로운 창 핸들 가져오기
            driver.switch_to.window(new_window)  # 새로운 창으로 전환

            time.sleep(3)
            # 상장일 가져오기 
            
            try:
                listed_time_value = driver.find_element(By.XPATH, '//*[@id="tab-contents"]/table[1]/tbody/tr[4]/td[2]').text  # XPath를 사용하여 텍스트 가져오기
                listed_time.append(listed_time_value)  # 상장일 리스트에 추가
                # 여기서 자주 에러 발생 -> 미처 로드 되기 전에 긁으려 해서 긁히지 않을 때가 있다.
                # 단, 시간이 늘어날수록 전체 스크래핑 시간도 증가함을 명심하자.
            except:
                print("상장일 찾던 중 StaleElementReferenceException 발생: 요소를 다시 찾습니다.")
                time.sleep(1)
                listed_time_value = driver.find_element(By.XPATH, '//*[@id="tab-contents"]/table[1]/tbody/tr[4]/td[2]').text  # XPath를 사용하여 텍스트 가져오기
                listed_time.append(listed_time_value)  # 상장일 리스트에 추가
            

            # 새로운 창에서 버튼 클릭하기 (예: 데이터 로드 버튼)
            load_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '/html/body/form/section/nav/ul/li[2]'))  # 데이터 로드 버튼의 XPATH로 변경
            )
            load_button.click()

            # 데이터 스크래핑
            data = []
            num_columns = 3

            time.sleep(1) # 또 기다려보기

            # 데이터 요소를 다시 찾기
            data_elements = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '/html/body/form/section/div/table[1]/tbody'))  # 테이블의 XPATH로 변경
            )

            for _ in range(3):  # 최대 3회 시도
                try:
                    for element in data_elements:
                        col_data = [cell.text for cell in element.find_elements(By.TAG_NAME, 'th')] # 행 인덱스
                        row_data = [cell.text for cell in element.find_elements(By.TAG_NAME, 'td')]  # 각 행의 데이터 가져오기
                    break  # 성공적으로 데이터를 가져오면 반복 종료
                except StaleElementReferenceException:
                    # 예외 발생 시 다시 시도
                    data_elements = driver.find_elements(By.XPATH, '/html/body/form/section/div/table[1]/tbody')  # 요소를 다시 찾기

            # 데이터프레임으로 변환
            data = [row_data[i:i + num_columns] for i in range(0, len(row_data), num_columns)] # 3열로 나누어 저장
            df = pd.DataFrame(data[1:], columns=[data[0][0], data[0][1], data[0][2]], index = col_data[1:]) # 형태 잡아주기
            row.append(df)

            # 새로운 창을 닫고 원래 창으로 돌아가기
            driver.close()  # 새로운 창 닫기
            driver.switch_to.window(original_window)  # 원래 창으로 전환

        except (ElementClickInterceptedException, StaleElementReferenceException, TimeoutException) as e:
            print(f"Error occurred at iteration {i}: {e}")
            break  # 오류 발생 시 반복문 종료

    # 다음 페이지로 넘어가기
    print(f'page {j} scrap complete!')

    if j == 10:
        break

#-------------
    button4 = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/section[2]/section/div/article/section[2]/div[1]/a[12]'))
    )
    
    try:
        button4.click()
    except StaleElementReferenceException:
        button4 = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/section[2]/section/div/article/section[2]/div[1]/a[12]'))
        )
        button4.click()

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/section[2]/section/div/article/section[1]/table/tbody/tr[1]/td[1]/a')))



# 드라이버 종료
driver.quit()

page 1 scrap complete!
page 2 scrap complete!
page 3 scrap complete!
page 4 scrap complete!
page 5 scrap complete!
page 6 scrap complete!
page 7 scrap complete!
page 8 scrap complete!
Error occurred at iteration 7: Message: 
Stacktrace:
	GetHandleVerifier [0x010B74A3+25091]
	(No symbol) [0x0103DC04]
	(No symbol) [0x00F1B373]
	(No symbol) [0x00F5F4DC]
	(No symbol) [0x00F5F65B]
	(No symbol) [0x00F9D8E2]
	(No symbol) [0x00F81F54]
	(No symbol) [0x00F9B49E]
	(No symbol) [0x00F81CA6]
	(No symbol) [0x00F531D5]
	(No symbol) [0x00F5435D]
	GetHandleVerifier [0x013B07C3+3142947]
	GetHandleVerifier [0x013C1A2B+3213195]
	GetHandleVerifier [0x013BC412+3191154]
	GetHandleVerifier [0x01158720+685184]
	(No symbol) [0x01046E1D]
	(No symbol) [0x01043E18]
	(No symbol) [0x01043FB6]
	(No symbol) [0x010366F0]
	BaseThreadInitThunk [0x75B47BA9+25]
	RtlInitializeExceptionChain [0x7786C0CB+107]
	RtlClearBits [0x7786C04F+191]

page 9 scrap complete!


In [79]:
# 잘 되었나 확인하기 -> 126이면 잘 다져온 것
print(len(corp_names), len(row), len(listed_time))

126 126 126


In [107]:
for i in range(len(row)):
    row[i] = row[i].reset_index()

In [109]:
# 이름-정보로 딕셔너리화
corp_financial_statement = dict(zip(corp_names, row))
corp_listed_date = dict(zip(corp_names, listed_time))

In [110]:
corp_listed_date['테크엔']

'2018-12-21'

In [111]:
corp_financial_statement[list(corp_financial_statement.keys())[112]]

,index,2021 년 (K-GAAP),2022 년 (K-GAAP),2023 년 (K-GAAP)
0,비유동자산(계),"3,020,245","2,484,387","3,688,697"
1,유형자산(계),"1,176,905","1,045,015","1,622,985"
2,리스자산,"250,658","146,359","743,471"
3,무형자산(계),"683,447","571,996","1,065,990"
4,기타의무형자산(계),"683,447","571,996","1,065,990"
5,장기투자자산,"322,098","356,319","393,141"
6,매출채권및기타채권,"468,327","250,690","293,389"
7,유동자산(계),"9,088,947","11,679,990","11,692,838"
8,재고자산(계),"4,286,033","5,244,044","4,079,900"
9,매출채권및기타채권,"2,848,227","2,644,150","2,490,347"


In [112]:
import json

# corp_financial_statement를 JSON으로 저장
corp_financial_statement_json = {key: df.to_dict(orient='records') for key, df in corp_financial_statement.items()}

# 딕셔너리 저장
with open('corp_financial_statement.json', 'w', encoding='utf-8') as json_file:
    json.dump(corp_financial_statement_json, json_file, ensure_ascii=False, indent=4)  # ensure_ascii=False는 한글을 제대로 저장하기 위함with open('data.json', 'w', encoding='utf-8') as json_file:

with open('corp_listed_date.json', 'w', encoding='utf-8') as json_file:
    json.dump(corp_listed_date, json_file, ensure_ascii=False, indent=4)

# KOSDAQ 버전
- 주의 : 오래걸린다.
- 총 118페이지로 이루어져 있음.

In [119]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException, ElementClickInterceptedException, TimeoutException
import time

row_KOSDAQ = []
corp_names_KOSDAQ = []
listed_time_KOSDAQ = []

# 웹 드라이버 경로 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 웹 페이지 열기
driver.get("https://kind.krx.co.kr/listinvstg/bzrsltdrft.do?method=searchBzRsltDrftMain")
time.sleep(3)

# 코스닥 시장 선택
button1 = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f'/html/body/section[2]/section/form/section/div/div[1]/table/tbody/tr/td[1]/input[3]'))  # 버튼의 XPATH로 변경
)
button1.click()
time.sleep(1)

# 코스닥 시장 검색
button2 = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f'/html/body/section[2]/section/form/section/div/div[3]/a[1]'))  # 버튼의 XPATH로 변경
)
button2.click()
time.sleep(1)

page_num = 150

# 특정 버튼을 기다리고 클릭하기
for j in range(1,page_num): #10
    time.sleep(3)
    for i in range(1, 16): #16
        try:
            # 각 기업명 누르기
            button3 = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f'/html/body/section[2]/section/div/article/section[1]/table/tbody/tr[{i}]/td[1]/a'))  # 버튼의 XPATH로 변경
            )
            button3.click()

            corp = button3.text
            corp_names_KOSDAQ.append(corp)

            # 새로운 창으로 전환하기
            WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))  # 새로운 창이 열릴 때까지 대기
            original_window = driver.current_window_handle  # 원래 창 핸들 저장
            new_window = [window for window in driver.window_handles if window != original_window][0]  # 새로운 창 핸들 가져오기
            driver.switch_to.window(new_window)  # 새로운 창으로 전환

            time.sleep(1)
            # 상장일 가져오기 
            
            try:
                listed_time_value = driver.find_element(By.XPATH, '//*[@id="tab-contents"]/table[1]/tbody/tr[4]/td[2]').text  # XPath를 사용하여 텍스트 가져오기
                listed_time_KOSDAQ.append(listed_time_value)  # 상장일 리스트에 추가
                # 여기서 자주 에러 발생 -> 미처 로드 되기 전에 긁으려 해서 긁히지 않을 때가 있다.
                # 단, 시간이 늘어날수록 전체 스크래핑 시간도 증가함을 명심하자.
            except:
                print("상장일 찾던 중 StaleElementReferenceException 발생: 요소를 다시 찾습니다.")
                time.sleep(1)
                listed_time_value = driver.find_element(By.XPATH, '//*[@id="tab-contents"]/table[1]/tbody/tr[4]/td[2]').text  # XPath를 사용하여 텍스트 가져오기
                listed_time_KOSDAQ.append(listed_time_value)  # 상장일 리스트에 추가
            
            time.sleep(1)

            # 새로운 창에서 버튼 클릭하기 (예: 데이터 로드 버튼)
            load_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '/html/body/form/section/nav/ul/li[2]'))  # 데이터 로드 버튼의 XPATH로 변경
            )
            load_button.click()

            # 데이터 스크래핑
            data = []
            num_columns = 3

            time.sleep(1) # 또 기다려보기

            # 데이터 요소를 다시 찾기
            data_elements = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '/html/body/form/section/div/table[1]/tbody'))  # 테이블의 XPATH로 변경
            )

            for _ in range(3):  # 최대 3회 시도
                try:
                    for element in data_elements:
                        col_data = [cell.text for cell in element.find_elements(By.TAG_NAME, 'th')]
                        row_data = [cell.text for cell in element.find_elements(By.TAG_NAME, 'td')]  # 각 행의 데이터 가져오기
                    break  # 성공적으로 데이터를 가져오면 반복 종료
                except StaleElementReferenceException:
                    # 예외 발생 시 다시 시도
                    data_elements = driver.find_elements(By.XPATH, '/html/body/form/section/div/table[1]/tbody')  # 요소를 다시 찾기

            # 데이터프레임으로 변환
            data = [row_data[i:i + num_columns] for i in range(0, len(row_data), num_columns)] # 3열로 나누어 저장
            df = pd.DataFrame(data[1:], columns=[data[0][0], data[0][1], data[0][2]], index = col_data[1:]) # 형태 잡아주기
            row_KOSDAQ.append(df)

            # 새로운 창을 닫고 원래 창으로 돌아가기
            driver.close()  # 새로운 창 닫기
            driver.switch_to.window(original_window)  # 원래 창으로 전환

        except (ElementClickInterceptedException, StaleElementReferenceException, TimeoutException) as e:
            print(f"Error occurred at iteration {i}: {e}")
            break  # 오류 발생 시 반복문 종료

    # 다음 페이지로 넘어가기
    print(f'page {j} scrap complete!')

    if j == page_num:
        break

    button4 = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/section[2]/section/div/article/section[2]/div[1]/a[12]'))
    ) # 다음 페이지로 가는 버튼 누르기 -> 이걸 조작하면...
    
    try:
        button4.click()
    except StaleElementReferenceException:
        button4 = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/section[2]/section/div/article/section[2]/div[1]/a[12]'))
        )
        button4.click()

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/section[2]/section/div/article/section[1]/table/tbody/tr[1]/td[1]/a')))



# 드라이버 종료
driver.quit() 

# 48페이지 스크래핑 중 에러 발생

page 1 scrap complete!
page 2 scrap complete!
page 3 scrap complete!
page 4 scrap complete!
page 5 scrap complete!
page 6 scrap complete!
page 7 scrap complete!
page 8 scrap complete!
page 9 scrap complete!
page 10 scrap complete!
page 11 scrap complete!
page 12 scrap complete!
page 13 scrap complete!
page 14 scrap complete!
page 15 scrap complete!
page 16 scrap complete!
page 17 scrap complete!
page 18 scrap complete!
page 19 scrap complete!
page 20 scrap complete!
page 21 scrap complete!
page 22 scrap complete!
page 23 scrap complete!
page 24 scrap complete!
page 25 scrap complete!
page 26 scrap complete!
page 27 scrap complete!
page 28 scrap complete!
page 29 scrap complete!
page 30 scrap complete!
page 31 scrap complete!
page 32 scrap complete!
page 33 scrap complete!
page 34 scrap complete!
page 35 scrap complete!
page 36 scrap complete!
page 37 scrap complete!
page 38 scrap complete!
page 39 scrap complete!
page 40 scrap complete!
page 41 scrap complete!
page 42 scrap complete!
p

ValueError: Length of values (5) does not match length of index (15)

In [120]:
print(len(listed_time_KOSDAQ), len(row_KOSDAQ), len(corp_names_KOSDAQ))

714 713 714


In [122]:
corp_names_KOSDAQ[713] # 이 놈이 문제다. -> 재무재표가 아니라 이상한 걸 주워옴!

'모니터랩'

In [128]:
col_data #모니터랩꺼 -> '재무정보' 누르는 게 제대로 실행이 되지 않은 듯 함

['한글명',
 '영문명',
 '표준코드',
 '종목코드',
 '설립일',
 '시장구분',
 '대표이사',
 '상장일',
 '자본금(한국원)',
 '종업원수',
 '결산월',
 '전화번호',
 '업종',
 '주요제품',
 '주소',
 '홈페이지']

In [124]:
row_data

['  모니터랩',
 'MONITORAPP',
 'KR7434480000',
 '434480',
 '2005-02-22',
 '코스닥 상장',
 '이광후',
 '2023-05-19',
 '1,225,330,000',
 '-  명',
 '12  월',
 '02-749-0799',
 '소프트웨어 개발 및 공급업',
 '웹방화벽,유해사이트솔루션,가시성SSL 솔루션, SASE플랫폼기반 SECaaS',
 '서울특별시 구로구 디지털로 306 대륭포스트타워 2차 1008호 ',
 '']

In [126]:
row_KOSDAQ[712]

,2021 년 (K-IFRS),2022 년 (K-IFRS),2023 년 (K-IFRS)
비유동자산(계),"13,321,113","13,696,811","18,416,620"
유형자산(계),"6,399,190","4,894,773","11,435,194"
리스자산,"265,497","2,196,902","2,803,566"
건설중인자산,"85,718",-,-
무형자산(계),"2,846,511","2,603,813","2,941,525"
기타의무형자산(계),"2,461,388","2,603,813","2,941,525"
장기투자자산,"380,240","626,126","1,825,762"
매출채권및기타채권,"505,432","1,341,420","1,438,601"
유동자산(계),"23,369,795","23,484,755","24,150,594"
재고자산(계),"163,912","141,531","155,977"


In [ ]:
for i in range(len(row_KOSDAQ)):
    row_KOSDAQ[i] = row_KOSDAQ[i].reset_index()

In [98]:
import json

# corp_financial_statement를 JSON으로 저장
corp_financial_statement_json = {key: df.to_dict(orient='records') for key, df in corp_financial_statement.items()}

# 딕셔너리 저장
with open('corp_financial_statement.json', 'w', encoding='utf-8') as json_file:
    json.dump(corp_financial_statement_json, json_file, ensure_ascii=False, indent=4)  # ensure_ascii=False는 한글을 제대로 저장하기 위함with open('data.json', 'w', encoding='utf-8') as json_file:

with open('corp_listed_date.json', 'w', encoding='utf-8') as json_file:
    json.dump(corp_listed_date, json_file, ensure_ascii=False, indent=4)

In [99]:
row_KOSDAQ[0]

,2021 년 (K-IFRS),2022 년 (K-IFRS),2023 년 (K-IFRS)
비유동자산(계),"16,870,835","7,753,391","7,665,768"
유형자산(계),"5,528,843","5,358,205","4,977,506"
리스자산,-,-,"112,415"
무형자산(계),"232,476","189,980","278,492"
기타의무형자산(계),"232,476","189,980","278,492"
장기투자자산,"8,893,785","167,013","171,170"
매출채권및기타채권,"378,430","57,800","82,505"
기타비금융자산,"135,375","135,375",-
유동자산(계),"38,720,318","42,467,558","47,693,886"
매출채권및기타채권,"6,906,535","6,108,152","5,529,164"
